<a href="https://colab.research.google.com/github/nittayac/DFEDATA6-EX1/blob/main/03_nlp_azure_PII-data-redaction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install azure-ai-textanalytics==5.1.0
!pip install azure-storage-blob

     |████████████████████████████████| 153 kB 12.5 MB/s 
     |████████████████████████████████| 85 kB 2.9 MB/s 
     |████████████████████████████████| 179 kB 27.7 MB/s 
     |████████████████████████████████| 41 kB 221 kB/s 
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 3.10.0.2
    Uninstalling typing-extensions-3.10.0.2:
      Successfully uninstalled typing-extensions-3.10.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.8.0 requires tf-estimator-nightly==2.8.0.dev2021122109, which is not installed.
arviz 0.11.4 requires typing-extensions<4,>=3.7.4.3, but you have typing-extensions 4.1.1 which is incompatible.
     |████████████████████████████████| 346 kB 18.3 MB/s 
     |████████████████████████████████| 3.6 MB 56.6 MB/s 


In [2]:
key = "0db5b0602c5142d1af19b3581859e017"
endpoint = "https://textanalysis20220310.cognitiveservices.azure.com/"

from azure.ai.textanalytics import TextAnalyticsClient
from azure.core.credentials import AzureKeyCredential

ta_credential = AzureKeyCredential(key)
client = TextAnalyticsClient(
            endpoint=endpoint, 
            credential=ta_credential)


In [34]:
from azure.storage.blob import BlobClient
# working with PRIVATE data-> in CV example, we took PUBLIC blob. 
#on left menu of STORAGE ACCOUNT-> find connection string below access key
constring = 'DefaultEndpointsProtocol=https;AccountName=textanalysis20220310;AccountKey=BzJHMqlB3x9gyNQIWaCyTUVj6z7cFIrR16tBn6ARHBfUzax0V9c32hjktJ4XlMv6hJiPnNI2NRtT+ASt37knZw==;EndpointSuffix=core.windows.net'
blob = BlobClient.from_connection_string(conn_str=constring, 
                                         container_name="textanalysis", 
                                         blob_name="pii.txt")
# blob-> written in binary
with open("pii.txt", "wb") as my_blob:
    blob_data = blob.download_blob()
    blob_data.readinto(my_blob)

In [35]:
# text-> read as text
mydata = open("pii.txt", "rt").read()
mydata

'My name is flow Pandey. My number is 083-2182865.\nI live at 221 Baker Street, Ghost Lake, Screamville.'

In [36]:
mydata = mydata.replace("\n","")
list1 = mydata.split('.')
sentokens =[]
for i in list1:
  i = i.strip()
  if len(i) > 0:
    sentokens.append(i)
print(sentokens)

['My name is flow Pandey', 'My number is 083-2182865', 'I live at 221 Baker Street, Ghost Lake, Screamville']


In [37]:


# Example method for detecting sensitive information (PII) from text 
def pii_recognition_example(client, mydata):
    documents = mydata
    response = client.recognize_pii_entities(documents, language="en")
    result = [doc for doc in response if not doc.is_error]
    for doc in result:
        print("Redacted Text: {}".format(doc.redacted_text))
        for entity in doc.entities:
            print("Entity: {}".format(entity.text))
            print("\tCategory: {}".format(entity.category))
            print("\tConfidence Score: {}".format(entity.confidence_score))
            print("\tOffset: {}".format(entity.offset))
            print("\tLength: {}".format(entity.length))


In [38]:
pii_recognition_example(client, sentokens)

Redacted Text: My name is flow ******
Entity: Pandey
	Category: Person
	Confidence Score: 0.46
	Offset: 16
	Length: 6
Redacted Text: My number is ***********
Entity: 083-2182865
	Category: PhoneNumber
	Confidence Score: 0.8
	Offset: 13
	Length: 11
Redacted Text: I live at *****************************************
Entity: 221 Baker Street, Ghost Lake, Screamville
	Category: Address
	Confidence Score: 0.84
	Offset: 10
	Length: 41
